# Variational Autoencoder - MNIST dataset

In [ ]:
import struct
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# pytorch
import torch
import torch.nn.functional as F
import torch.utils.data as Data
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms


In [ ]:
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

---
* Utilities

In [ ]:
def plot_digit(x):
    x = x.data.numpy().reshape(data1.shape[1],data1.shape[2])
    plt.imshow(x,cmap='gray_r')
    plt.show()
    
def generate_samples(encoder,decoder,mb_size):
    for _, batch in enumerate(loader):
        X = Variable(batch[0]).to(device)
        X_size = X.shape[0]
        X = X.reshape(X_size,28*28).to(device)
        z = Variable(torch.randn(X_size, Z_dim)).to(device)
        z_mu, z_var = encoder(X)
        z = sample_z(z_mu, z_var,X_size)
        X_sample = decoder(z)
    return X_sample[:mb_size]

def plot_n_random_digits(x,labels=None,n_images=9,title='Digits'):
    """Ate 20 imagens"""
    indices = np.random.choice(x.shape[0],size=n_images)
    x = x[indices].data.numpy()
    if isinstance(labels, torch.IntTensor):
        sample_labels = labels[indices].data.numpy()
    else: sample_labels = ['None']*len(indices)
    x = x.reshape(n_images,data1.shape[1],data1.shape[2])
    plt.clf()
    plt.style.use('seaborn-muted')
    fig, axes = plt.subplots(2,10, figsize=(15,3), sharex=True, sharey=True,
                             subplot_kw=dict(adjustable='box', aspect='equal'))
    for i in range(n_images):
        subplot_row = i//10
        subplot_col = i%10  
        ax = axes[subplot_row, subplot_col]
        plottable_image = x[i,:]
        ax.imshow(plottable_image, cmap='gray_r')
        ax.set_title('Digit Label: {}'.format(sample_labels[i]))
        ax.set_xbound([0,28])
    plt.tight_layout()
    plt.show()

def print_label(label):
    chosen_label = label
    batch = 1000
    y_sample = np.zeros(shape=[batch, y_dim])
    y_sample[:,chosen_label] = 1
    y_sample_tc = torch.Tensor(y_sample).type(torch.IntTensor)
    plot_n_random_digits(G(Variable(torch.randn(batch, Z_dim)).cuda(),
                         y_sample_tc.type(torch.FloatTensor).cuda()).cpu(),
                         labels=torch.max(y_sample_tc,dim=1)[1].type(torch.IntTensor),
                         n_images=20,title='conditional_gan_generated_{}_{}_'.format(chosen_label,epochs))

In [ ]:
datasets.MNIST('./data', train=True, download=True)

In [ ]:
file_images = '/home/ninja/MNIST/train-images.idx3-ubyte'
file_labels = '/home/ninja/MNIST/train-labels.idx1-ubyte'

In [ ]:
with open(file_images,'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data1 = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    data1 = data1.reshape((size, nrows, ncols))

data1.shape

In [ ]:
plt.imshow(data1[4,:,:], cmap='gray')
plt.show()

In [ ]:
data = data1.reshape(data1.shape[0],data1.shape[1]*data1.shape[2])
data.shape

In [ ]:
with open(file_labels,'rb') as f:
    labels = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))

labels = labels[8:]
labels.shape

---

# Data prep
    1- Escalar os dados entre zero e 1
    2- Transformar os dados (X e y) em um tensor do tipo float
        Ex: torch.Tensor(x).type(torch.FloatTensor)
    3- Verificar se as dimensões e tipo estão corretas:
        X_dim = (60k,784), y_dim = (60k,1)

In [ ]:
# 1
scaler = 
data = scaler.fit_transform(data)
# 2
X_tc = 
y_tc = 
# 3
print(X_tc.shape, X_tc.type(),y_tc.shape, y_tc.type())

    4- Plotar uma observação qualquer

In [ ]:
plot_digit(    )

    5- Plotar 20 dígitos

In [ ]:
plot_n_random_digits(   ,n_images=   )

## DataLoader
    1- Definir um batch_size
    2- Determinar as dimensões usadas nas redes neurais
    3- Definir quantas camadas serão usadas tb seus tamanhos

In [ ]:
# 1
mb_size = 
# 2
Z_dim = 
X_dim = 
y_dim = 
# 3
a_dim, b_dim, ... = [  ,  , ...]

    4- Passar os dados para o DataLoader

In [ ]:
torch_dataset = Data.TensorDataset(...)
loader = Data.DataLoader(
        dataset = torch_dataset,
        batch_size = mb_size,
        shuffle=True,
        pin_memory=True,
        num_workers=12
        )

## Construindo as redes neurais
    1- Ensira o número de camadas escolhidas no construtor da classe Encoder usando camadas lineares
        Ex: torch.nn.Linear(dimensão de entrada, dimensão de saída)
    2- No método forward use uma função relu em todas as camadas em sequência
        Ex: h = F.relu(self.hidden1(X))
    3- O Encoder deve ter duas saídas

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self,ngpu):
        super(Encoder,self).__init__()
        self.ngpu = ngpu
        # 1
        self.hidden1 = torch.nn.Linear(X_dim, a_dim)
        self.hidden2 = ...
        ...
    
    def forward(self,X):
        # 2
        h = F.relu(self.hidden1(X))
        h = F.relu(self.hidden2(h))
        ...
        # 3
        mu = 
        log_sigma = 
        return mu, log_sigma

    4- Construa a função que faz a amostragem de z
        Ex: epsilon = Variable(torch.randn(mb_size, Z_dim)).to(device)
            z = (mu + torch.exp(log_sigma / 2) * epsilon).to(device)

In [ ]:
def sample_z(mu, log_sigma,mb_size):
    epsilon = Variable(torch.randn(mb_size, Z_dim)).to(device)
    return ...

    6- Construa o Decoder. Ele é quase o inverso do Decoder com a diferença que recebe uma entrada ao invés de duas
    7- Aplique a ativação sigmoid na saída do Decoder

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self,ngpu):
        super(Decoder,self).__init__()
        self.ngpu = ngpu
        # 6
        self.hidden1 = torch.nn.Linear(Z_dim, ...)
    
    def forward(self,z):
        h = F.relu(self.hidden1(z))
        ...
        # 7
        out = torch.sigmoid(...)
        return out   

## Instanciando e definindo o Otimizador
    1- Instancie o Encoder e o Decoder como no exemplo abaixo

In [ ]:
rede1 = Rede1(ngpu).to(device)
if (device.type == 'cuda') and (ngpu > 1):
    rede1 = nn.DataParallel(rede1, list(range(ngpu)))

    2- Defina uma taxa de aprendizado
    3- Crie uma lista com os parâmetros da duas redes e passe par

In [ ]:
lr = 
params = list(rede1.parameters()) + list(rede1.parameters())
optimizer = optim.Adam(params, lr=lr, betas=(0.9,0.999))

    4- Aplique a inicialicação de pesos nas duas redes

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight).to(device)
        m.bias.data.fill_(0.001)

#Inicializar pesos das redes
rede1.apply(init_weights)
...

# lista para guardar as informações das perdas
loss_his = []

    5- Se já tiver um modelo salvo pode carregar os pessos com o código abaixo

In [ ]:
#load saved networks
# encoder.load_state_dict(torch.load('models/encoder_mnist.pt'))
# decoder.load_state_dict(torch.load('models/decoder_mnist.pt'))

    6- Verifique se as redes saíram como você esperava

In [ ]:
print(rede1)
print(rede2)

## Treinando
    1- Defina um número de épocas
    2- Defina o X_size como o dimensão zero de X
    3- Passe o X para o Encoder devolver suas saídas
    4- Faça a amostragem de z
    5- Passe z para o Decoder retornar X
    6- Calcule o erro de reconstrução com a função de perda:
        F.binary_cross_entropy(X_falso, X_real, reduction='sum')
    7- Calcule a divergência KL utilizando sua forma fechada:
        0.5*sum(exp(z_var) + z_mu**2 - 1. - z_var)
    8- Some as duas para obter a perda total
    9- Calcule os gradientes usando loss.backward()
    10- Atualize o otimizador usando optimizer.step() e zere os gradientes para a próxima iteração com optimizer.zero_grad()

In [ ]:
%%time
# 1
epochs = 

for epoch in range(epochs):

    for step, batch_x in enumerate(loader):
        X = Variable(batch_x[0]).to(device)
        # 2
        X_size = 
        # 3
        z_mu, z_var = encoder(  )
        # 4
        z = sample_z(   ,   ,X_size)
        # 5
        X_sample = decoder(   )
        
        # 6
        recon_loss = ...
        # 7
        kl_loss = torch.mean(    * torch.sum(       , dim=1))
        # 8
        loss = ...
        # 9
        ...
        # 10
        ...
        ...

        # Recolher dados das funções de perda
        loss_his.append(loss.data)

    # Mostrar dados ao longo das epochs
    if epoch % 10 == 0:
        print('Epoch-{}| Average loss: {:.5f}'.format(epoch, loss.mean().data))

## Resultados
    1- Verifique a evolução da perda

In [ ]:
plt.plot(loss_his)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show();

    2- Salve o modelo se quiser

In [ ]:
# Save model
# torch.save(encoder.state_dict(), 'models/encoder_mnist.pt')
# torch.save(decoder.state_dict(), 'models/decoder_mnist.pt')

    3- Gera alguns dados e comparar com os dados verdadeiros

In [ ]:
X_fake = generate_samples(encoder,decoder,mb_size=20)

In [ ]:
plot_n_random_digits(X_fake.cpu(),labels=None,n_images=20,title='Digits')

In [ ]:
plot_n_random_digits(X_tc,n_images=20)

# Bônus

In [ ]:
def generate_full_sample():
    X_samples = torch.Tensor().to(device)
    Z_samples = torch.Tensor().to(device)
    for step, batch in enumerate(loader):
        X = Variable(batch[0]).to(device)
        z_mu, z_var = encoder(X)
        z = sample_z(z_mu, z_var,X.shape[0])
        X_samples = torch.cat([X_samples,decoder(z)],dim=0)
        Z_samples = torch.cat([Z_samples,z],dim=0)
    return X_samples,Z_samples

In [ ]:
X_samples,Z_samples = generate_full_sample()
X_samples.shape,Z_samples.shape

In [ ]:
sample_size = 1000
X_fake = X_samples[:sample_size].cpu().data.numpy()
X_real = X_tc[:sample_size].cpu().data.numpy()
Z = Z_samples[:sample_size].cpu().data.numpy()

## Visualizando o "espaço de features"

In [ ]:
import umap

In [ ]:
%%time
embedding_real = umap.UMAP(n_neighbors=25,min_dist=0.3,n_epochs=2000,
                      metric='euclidean').fit_transform(X_real)
embedding_fake = umap.UMAP(n_neighbors=25,min_dist=0.3,n_epochs=2000,
                      metric='euclidean').fit_transform(X_fake)

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(x=embedding_real[:,0],y=embedding_real[:,1],label="real")
plt.scatter(x=embedding_fake[:,0],y=embedding_fake[:,1],label="fake")
plt.legend(loc=0)
plt.grid(True)
plt.show()

In [ ]:
digit = 0
plt.figure(figsize=(10,6))
embedding_real_ = embedding_real[np.where(labels[:sample_size]==digit)]
embedding_fake_ = embedding_fake[np.where(labels[:sample_size]==digit)]
labels_ = [np.where(labels[:sample_size]==0)]
plt.scatter(x=embedding_real_[:,0],y=embedding_real_[:,1],label="real")
plt.scatter(x=embedding_fake_[:,0],y=embedding_fake_[:,1],label="fake")
plt.legend(loc=0)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
for i in range(10):
    embedding_real_ = embedding_real[np.where(labels[:sample_size]==i)]
    labels_ = [np.where(labels[:sample_size]==0)]
    plt.scatter(x=embedding_real_[:,0],y=embedding_real_[:,1],label=i)
plt.legend(loc=0)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
for i in range(10):
    embedding_fake_ = embedding_fake[np.where(labels[:sample_size]==i)]
    labels_ = [np.where(labels[:sample_size]==0)]
    plt.scatter(x=embedding_fake_[:,0],y=embedding_fake_[:,1],label=i)
plt.legend(loc=0)
plt.grid(True)
plt.show()

## Visualizando o "espaço latente"

In [ ]:
%%time
embedding_Z = umap.UMAP(n_neighbors=25,min_dist=0.3,n_epochs=2000,
                      metric='euclidean').fit_transform(Z)

In [ ]:
plt.figure(figsize=(10,6))
for i in range(10):
    embedding_Z_ = embedding_Z[np.where(labels[:sample_size]==i)]
    labels_ = [np.where(labels[:sample_size]==0)]
    plt.scatter(x=embedding_Z_[:,0],y=embedding_Z_[:,1],label=i)
plt.legend(loc=0)
plt.grid(True)
plt.show()

## Métricas topológicas

In [ ]:
import gs
import gudhi as gd

* Geometry Score

In [ ]:
def geometry_score(X_real,X_fake,L_0=64, gamma=1.0/128,  i_max=100, n=500):
    rlt_fake = gs.rlts(X_fake, L_0=L_0, gamma=gamma,  i_max=i_max, n=n)
    rlt_real = gs.rlts(X_real, L_0=L_0, gamma=gamma,  i_max=i_max, n=n)
    return gs.geom_score(rlt_real, rlt_fake), np.mean(rlt_real, axis=0), np.mean(rlt_fake, axis=0)

In [ ]:
%%time
geo_score, mrlt_real, mrlt_fake = geometry_score(X_real,X_fake,L_0=64, gamma=1.0/128,  i_max=200, n=1000)

In [ ]:
gs.fancy_plot(mrlt_fake, label='MRLT of fake data',color='blueviolet')
gs.fancy_plot(mrlt_real, label='MRLT of real data',color='limegreen')
plt.title('Geometry Score: {:.3f}'.format(geo_score))
plt.xlim([0, 110])
plt.legend()

* Persistence Homology

In [ ]:
%%time
X_real_ripscomplex = gd.RipsComplex(points=X_real,max_edge_length=7.0) 
Rips_simplex_tree_X_real = X_real_ripscomplex.create_simplex_tree(max_dimension=2)
BarCodes_Rips0_real = Rips_simplex_tree_X_real.persistence()

In [ ]:
%%time
X_fake_ripscomplex = gd.RipsComplex(points=X_fake,max_edge_length=7.0) 
Rips_simplex_tree_X_fake = X_fake_ripscomplex.create_simplex_tree(max_dimension=2)
BarCodes_Rips0_fake = Rips_simplex_tree_X_fake.persistence()

In [ ]:
# real data
plot = gd.plot_persistence_barcode(BarCodes_Rips0_real)
plot.show()

In [ ]:
# fake data
plot = gd.plot_persistence_barcode(BarCodes_Rips0_fake)
plot.show()

In [ ]:
for i in range(3):
    print(f"Real data: Number o holes in {i} dimension: {len(Rips_simplex_tree_X_real.persistence_intervals_in_dimension(i))}")
    print(f"Fake data: Number o holes in {i} dimension: {len(Rips_simplex_tree_X_fake.persistence_intervals_in_dimension(i))}")

In [ ]:
gd.plot_persistence_diagram(BarCodes_Rips0_real);

In [ ]:
gd.plot_persistence_diagram(BarCodes_Rips0_fake);